In [23]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
import requests
from collections import defaultdict

In [4]:
property_metadata.head()

,Unnamed: 0,name,cost_text,coordinates,desc,beds,baths,parkings,weekly_rent
0,https://www.domain.com.au/904-265-exhibition-s...,904/265 Exhibition Street Melbourne VIC 3000,$850 Per Week,"[-37.8095116, 144.9691204]",Darren Krause,2.0,2.0,1.0,850.0
1,https://www.domain.com.au/210-422-collins-stre...,210/422 Collins Street Melbourne VIC 3000,$475 Per Week,"[-37.8170971, 144.9601487]",Annalise Zamparo,1.0,1.0,NaN,475.0
2,https://www.domain.com.au/1902-200-spencer-str...,1902/200 Spencer Street Melbourne VIC 3000,$630 pw,"[-37.816228, 144.9532465]",Paola Faba,2.0,1.0,1.0,630.0
3,https://www.domain.com.au/312b-399-bourke-stre...,312B/399 Bourke Street Melbourne VIC 3000,$450,"[-37.8147259, 144.9621291]",Natalie Subotsch,1.0,1.0,NaN,450.0
4,https://www.domain.com.au/3313-228-la-trobe-st...,3313/228 La Trobe Street Melbourne VIC 3000,$900,"[-37.8096052, 144.962371]",Paula Tran,2.0,1.0,2.0,900.0


In [5]:
# Your property data
property_metadata = pd.read_csv("../data/raw/properties.csv")

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
property_metadata['coordinates'] = property_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in property_metadata['coordinates']]
names = property_metadata['name'].tolist()
cost = property_metadata['weekly_rent'].tolist()
beds = property_metadata['beds'].tolist()
baths = property_metadata['baths'].tolist()
parkings = property_metadata['parkings'].tolist()

# Convert to GeoDataFrame
property = gpd.GeoDataFrame({'name': names, 'geometry': coords, 'cost': cost, 'beds': beds, 'baths': baths, 'parkings': parkings})


In [11]:
# Assuming property_gdf and victoria_gdf are already defined as per your provided code...
victoria_gdf = gpd.read_file('../data/Statistical_area_level2/victoria.geojson')

# Perform a spatial join
property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_3669/2101387466.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")


In [14]:
selected_columns = ['name', 'geometry', 'cost', 'beds', 'baths', 'parkings', 'SA2_CODE21', 'SA2_NAME21']
property_SA2 = property_SA2[selected_columns]

In [16]:
PTV_metro = gpd.read_file('../data/external_SA2/PTV/PTV_metro/PTV_METRO_TRAIN_STATION.shp')
PTV_regional = gpd.read_file("../data/external_SA2/PTV/PTV_regional/PTV_REGIONAL_TRAIN_STATION.shp")

In [20]:
PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)
PTV_station = PTV_station.drop_duplicates(subset='STOP_NAME', keep='first')

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_3669/1230334281.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)


In [64]:
def euclidean_distance(point1, point2):
    return ((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2) ** 0.5

def get_distance(first_point, second_point):
    ORS_BASE_URL = "http://localhost:8080/ors/v2/directions/driving-car"
    url = f'{ORS_BASE_URL}?start={first_point[0]},{first_point[1]}&end={second_point[0]},{second_point[1]}'
    response = requests.get(url)
    json_response = response.json()
    
    if 'error' not in json_response:
        try:
            return json_response['features'][0]['properties']['summary']['duration'] / 60
        except KeyError: 
            return float('inf')
    return float('inf')


def find_proximity(property_df, facility_df, name):
    # This will hold the name of the nearest facility for each property
    nearest_facilities = []

    # Loop through properties
    for idx1, prop_row in property_df.iterrows():
        distances = {}  # Using a dictionary to map distances to facility point
        prop_point = prop_row['geometry'].coords[0]
        
        # Loop through facilities
        for idx2, fac_row in facility_df.iterrows():
            fac_point = fac_row['geometry'].coords[0]
            # Compute the Euclidean distance
            dist = euclidean_distance(prop_point, fac_point)
            distances[dist] = fac_point

        # Finding the facility name corresponding to the shortest distance
        min_distance = min(distances.keys())
        nearest_facilities.append(get_distance(prop_point, distances[min_distance]))

    property_df['Nearest_{}'.format(name)] = nearest_facilities

    return property_df

In [34]:
property_station = find_proximity(property_SA2, PTV_station, 'station')

In [35]:
property_station.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.6452
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.4319
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,NaN,206041503,Melbourne CBD - East,2.1035
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,1.0761
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,0.3607


In [37]:
park = gpd.read_file('../data/external_SA2/Park.geojson')

In [39]:
property_station_park = find_proximity(property_station, park, 'park')

In [40]:
property_station_park.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.6452,1.7146
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.4319,2.7503
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,NaN,206041503,Melbourne CBD - East,2.1035,2.8712
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,1.0761,1.6096
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,0.3607,2.6702


In [41]:
shop = gpd.read_file('../data/external_SA2/Mall, Shopping Centre & Department Store.geojson')

In [43]:
property_station_park_shop = find_proximity(property_station_park, shop, 'shop')

In [44]:
property_station_park_shop.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.6452,1.7146,0.7007
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.4319,2.7503,1.1439
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,NaN,206041503,Melbourne CBD - East,2.1035,2.8712,1.2648
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,1.0761,1.6096,0.9307
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,0.3607,2.6702,1.0638


In [45]:
hospital = gpd.read_file('../data/external_SA2/Hospital.geojson')

In [47]:
property_station_park_shop_hospital = find_proximity(property_station_park_shop, hospital, 'hospital')

In [50]:
property_station_park_shop_hospital.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop,Nearest_hospital
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.6452,1.7146,0.7007,2.1130
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.4319,2.7503,1.1439,2.4385
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,NaN,206041503,Melbourne CBD - East,2.1035,2.8712,1.2648,2.7744
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,1.0761,1.6096,0.9307,2.0254
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,0.3607,2.6702,1.0638,2.2120


In [51]:
school_df = pd.read_csv("../data/external_SA2/complete_school_location.csv", encoding='latin-1')
school_df['coordinates'] = school_df.apply(lambda row: [row['Y'], row['X']], axis=1)

In [52]:
school_df.head()

,geometry,X,Y,School_Type,School_Name,Postal_Town,coordinates
0,POINT (145.3643874 -38.084432),145.364387,-38.084432,college,Hillcrest Christian College (Ayr Hill Campus),NaN,"[-38.084432, 145.3643874]"
1,POINT (-96.9814539 28.8155588),-96.981454,28.815559,college,Victoria College,NaN,"[28.8155588, -96.9814539]"
2,POINT (143.858787 -37.5611479),143.858787,-37.561148,college,Arts Academy,NaN,"[-37.5611479, 143.858787]"
3,POINT (144.972154 -37.8103129),144.972154,-37.810313,university,James Cook University,NaN,"[-37.8103129, 144.972154]"
4,POINT (145.1490967 -37.8768613),145.149097,-37.876861,college,Huanya Cultural Training Centre,NaN,"[-37.8768613, 145.1490967]"


In [53]:
# Your property data
school_metadata = school_df

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
#school_metadata['coordinates'] = school_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in school_metadata['coordinates']]
names = school_metadata['School_Type'].tolist()

# Convert to GeoDataFrame
school = gpd.GeoDataFrame({'name': names, 'geometry': coords})

In [ ]:
school_SA2 = gpd.sjoin(school, victoria_gdf, how="inner", op="within")

In [ ]:
school_SA2.head()

In [60]:
property_station_park_shop_hospital_school = find_proximity(property_station_park_shop_hospital, school_SA2, 'school')

In [61]:
property_station_park_shop_hospital_school.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop,Nearest_hospital,Nearest_school
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.6452,1.7146,0.7007,2.1130,0.3461
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.4319,2.7503,1.1439,2.4385,0.5171
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,NaN,206041503,Melbourne CBD - East,2.1035,2.8712,1.2648,2.7744,0.2480
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,1.0761,1.6096,0.9307,2.0254,0.4471
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,0.3607,2.6702,1.0638,2.2120,0.4370


In [62]:
supermarket = gpd.read_file('../data/external_SA2/Supermarket.geojson')

In [65]:
property_station_park_shop_hospital_school_supermarket = find_proximity(property_station_park_shop_hospital_school, supermarket, 'supermarket')

In [66]:
property_station_park_shop_hospital_school_supermarket.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop,Nearest_hospital,Nearest_school,Nearest_supermarket
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.6452,1.7146,0.7007,2.1130,0.3461,0.743333
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.4319,2.7503,1.1439,2.4385,0.5171,2.190000
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,NaN,206041503,Melbourne CBD - East,2.1035,2.8712,1.2648,2.7744,0.2480,2.823333
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,1.0761,1.6096,0.9307,2.0254,0.4471,2.193333
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,0.3607,2.6702,1.0638,2.2120,0.4370,1.968333


In [67]:
property_station_park_shop_hospital_school_supermarket.to_csv("../data/aggregated_dataset.csv")